In [1]:
from gurobipy import *
import numpy as np

In [2]:
#Indices
t_array = np.arange(0,6)
m_array = np.arange(0,8)
k_array = np.arange(0, 20)
i_array = np.arange(0, 10)
j_array = np.arange(0, 3)

In [3]:
oi = Model()

Restricted license - for non-production use only - expires 2024-10-28


In [4]:
#Variables
x = oi.addVars(((i,m) for i in i_array for m in m_array),vtype = GRB.BINARY)
e = oi.addVars(((k,i) for k in k_array for i in i_array),vtype = GRB.BINARY)
b = oi.addVars(((i) for i in i_array),vtype = GRB.BINARY)
y = oi.addVars(((k) for k in k_array),vtype = GRB.BINARY)
o = oi.addVars(((i,k) for i in i_array for k in k_array),vtype = GRB.CONTINUOUS)
w = oi.addVars(((i) for i in i_array),vtype = GRB.BINARY)
profit = oi.addVars(GRB.CONTINUOUS)

In [5]:
#Matrices
R_i =np.array([250, 1800, 1050, 3550, 550, 3600, 700, 3650, 1350, 2800])
et_tk = np.array([
    [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]])
L = 2
p = 1500
H = 280
G_j = np.array([10000, 50000, 800000])
S_t = np.array([8000, 16000, 24000, 32000, 40000, 48000])
ub_ti = np.array([
    [2, 2, 2, 3, 3, 3, 3, 3, 2, 3],
    [3, 2, 3, 2, 3, 3, 3, 2, 2, 2],
    [2, 2, 2, 3, 2, 2, 2, 2, 3, 2],
    [3, 2, 3, 2, 2, 2, 3, 2, 3, 3],
    [2, 3, 2, 2, 2, 3, 3, 2, 3, 3],
    [2, 3, 3, 2, 3, 2, 2, 2, 3, 3]])
lb_ti = np.array([
    [1, 1, 1, 1, 1, 1, 1, 0, 1, 0],
    [0, 1, 0, 1, 0, 0, 0, 1, 1, 0],
    [1, 0, 1, 1, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 1, 1, 0, 1, 0, 1, 1],
    [0, 1, 1, 0, 1, 1, 0, 0, 1, 1],
    [0, 1, 1, 0, 0, 1, 0, 0, 1, 1]])
d_ij = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 0, 0],
    [0, 0, 1],
    [1, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [0, 0, 1]])
a_im = np.array([
    [0, 0, 0, 0, 1, 0, 1, 1],
    [1, 1, 1, 0, 1, 0, 0, 1],
    [0, 1, 1, 1, 0, 1, 1, 1],
    [1, 1, 0, 0, 1, 1, 0, 0],
    [0, 0, 0, 0, 1, 1, 0, 0],
    [1, 0, 0, 1, 1, 1, 1, 1],
    [0, 0, 1, 1, 1, 0, 1, 1],
    [1, 0, 0, 0, 1, 1, 1, 0],
    [0, 0, 1, 0, 1, 1, 1, 1],
    [1, 0, 1, 1, 0, 1, 0, 1]])

In [6]:
#Constraints
for k in k_array:
    oi.addConstr(quicksum(e[k,i] for i in i_array)>=1)

In [7]:
for k in k_array:
    oi.addConstr(quicksum(e[k,i] for i in i_array)<=5)

In [8]:
for i in i_array:
    for t in t_array:
        oi.addConstr((quicksum(e[k,i] * et_tk[t,k] for k in k_array)<=ub_ti[t,i]))

In [9]:
for i in i_array:
    for t in t_array:
        oi.addConstr((quicksum(e[k,i] * et_tk[t,k] for k in k_array)>=lb_ti[t,i]))

In [10]:
for m in m_array:
    oi.addConstr(quicksum(x[i,m] for i in i_array)>=1)

In [11]:
for i in i_array:
    oi.addConstr(quicksum(x[i,m] for m in m_array)<=1)

In [12]:
oi.addConstr(quicksum(x[i,0] - x[i,1] for i in i_array)<=L)

<gurobi.Constr *Awaiting Model Update*>

In [13]:
oi.addConstr(quicksum(x[i,1] - x[i,0] for i in i_array)<=L)

<gurobi.Constr *Awaiting Model Update*>

In [14]:
for k in k_array:
    oi.addConstr(quicksum(o[i,k] for i in i_array)+y[k]*8*30<= H)

In [15]:
for k in k_array:
    oi.addConstr(y[k] <= quicksum(e[k,i] for i in i_array))

In [16]:
for k in k_array:
    oi.addConstr(10*y[k]>=quicksum(e[k,i] for i in i_array))

In [17]:
for k in k_array:
    for i in i_array:
        oi.addConstr(o[i,k]<=H*e[k,i])

In [18]:
for i in i_array:
    oi.addConstr(quicksum(x[i,m] for m in m_array)>=b[i])

In [19]:
for i in i_array:
    for m in m_array:
        oi.addConstr(x[i,m]<=a_im[i,m])

In [20]:
for i in i_array:
    oi.addConstr(quicksum(8*30*e[k,i] + o[i,k]*d_ij[i,2] for k in k_array) >= (R_i[i]*b[i]))

In [21]:
oi.addConstr(x[0,0]>=w[0])
oi.addConstr(x[1,0]>=w[0])
oi.addConstr(x[0,0]+x[1,0]-1<=w[0])
oi.addConstr(1-w[0]<=x[2,2])

<gurobi.Constr *Awaiting Model Update*>

In [22]:
for i in i_array:
    oi.addConstr((1-b[i])*d_ij[i,1]<=w[1])

In [23]:
for i in i_array:
    oi.addConstr(1-w[1]>=(1-b[i])*d_ij[i,2])

In [24]:
for i in i_array:
    for k in k_array:
        oi.addConstr(o[i,k] >= 0)

In [25]:
oi.setObjective(quicksum(d_ij[i,j]*b[i]*G_j[j] for i in i_array for j in j_array)-quicksum(p*(1-b[i])for i in i_array)-quicksum(e[k,i]*S_t[t]*et_tk[t,k] for i in i_array for t in t_array for k in k_array)-quicksum(o[i,k]*50 for i in i_array for k in k_array),GRB.MAXIMIZE)

In [26]:
oi.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11300H @ 3.10GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 764 rows, 521 columns and 2756 nonzeros
Model fingerprint: 0x3256ad11
Variable types: 201 continuous, 320 integer (320 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [5e+01, 8e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -885500.0000
Presolve removed 536 rows and 230 columns
Presolve time: 0.00s
Presolved: 228 rows, 291 columns, 1210 nonzeros
Variable types: 80 continuous, 211 integer (211 binary)
Found heuristic solution: objective -818000.0000

Root relaxation: objective 1.328664e+06, 143 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth Int